In [1]:
import numpy as np
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

word_count = 8000
max_review_length = 500 
embedding_dim = 30

#--------------loading pre-processed train and test data--------------
def load_data():
    x_train = np.load('x_train_reviews.npy')
    x_test = np.load('x_test_reviews.npy')
    y_train = np.load('y_train_labels.npy')
    y_test = np.load('y_test_labels.npy')

    #converting labels to binary, not one-hot encoded
    y_train = np.argmax(y_train, axis=1)
    y_test = np.argmax(y_test, axis=1)

    return x_train, x_test, y_train, y_test

#--------------lstm model architecture--------------
def create_model(word_count, max_review_length, embedding_dim):

    model = Sequential()
    
    model.add(Embedding(input_dim=word_count, output_dim=embedding_dim, input_length=max_review_length))
    model.add(LSTM(100))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

#--------------training lstm model--------------
def train_and_evaluate(model, x_train, y_train, x_test, y_test, epochs=5, batch_size=128):
    
    history = model.fit(
        x_train, y_train,
        validation_data=(x_test, y_test),
        epochs=epochs,
        batch_size=batch_size
    )

    #predicting the labels for the test set
    y_pred_prob = model.predict(x_test)
    y_pred = (y_pred_prob >= 0.5).astype(int)

    #calculating accuracy, precision, and recall
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')

    return model


x_train, x_test, y_train, y_test = load_data()
model = create_model(word_count, max_review_length, embedding_dim)
model = train_and_evaluate(model, x_train, y_train, x_test, y_test)
model.save('LSTM_sentiment_analysis_model.h5')


Epoch 1/5


D:\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


196/196 ━━━━━━━━━━━━━━━━━━━━ 203s 1s/step - accuracy: 0.6629 - loss: 0.6127 - val_accuracy: 0.8338 - val_loss: 0.3775
Epoch 2/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 223s 1s/step - accuracy: 0.8814 - loss: 0.2962 - val_accuracy: 0.8651 - val_loss: 0.3161
Epoch 3/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 235s 1s/step - accuracy: 0.9206 - loss: 0.2137 - val_accuracy: 0.8741 - val_loss: 0.2985
Epoch 4/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 227s 1s/step - accuracy: 0.9401 - loss: 0.1728 - val_accuracy: 0.8722 - val_loss: 0.3150
Epoch 5/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 254s 1s/step - accuracy: 0.9489 - loss: 0.1428 - val_accuracy: 0.8689 - val_loss: 0.3719
782/782 ━━━━━━━━━━━━━━━━━━━━ 90s 114ms/step


Accuracy: 0.8689
Precision: 0.8937
Recall: 0.8374


In [2]:
model = load_model('LSTM_sentiment_analysis_model.h5')

#--------------analyzing sentiment of user review--------------
def analyze_review(review):
   
    #loading tokenizer to pre-process input review 
    with open('tokenizer.pkl', 'rb') as f:
        tokenizer = pickle.load(f)
        
    sequences = tokenizer.texts_to_sequences([review])
    padded_sequences = pad_sequences(sequences, maxlen=max_review_length)

    #predicting user review label
    prediction = model.predict(padded_sequences)[0][0]
    
    label = 'positive' if prediction >= 0.5 else 'negative'
    return label

while True:
    input_prompt = input("Enter a review (or type 'exit' to quit): ")
    if input_prompt.lower() == 'exit':
        print("Exited sentiment analysis.")
        break
    predicted_label = analyze_review(input_prompt)
    print(f'The review is {predicted_label}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
The review is negative
Exited sentiment analysis.
